Preprocesado:
<ol><li> Juntar título y descripción</li>
<li> Eliminar textos cortos (menos o igual a dos palabras)</li>
<li>Minúsculas</li>
<li>sin signos de puntuación</li>
<li> sólo letras</li>
<li>sin stopwords</li>
<li>Lematizar</li>
<li> Quitar acentos</li>
<li>Eliminar tokens de 1 caracter</li>
<li> Eliminar palabras muy frecuentes o muy raras</li></ol>

In [21]:
#!pip install numpy==1.26.4
#!pip install spacy
#!python -m spacy download es_core_news_sm 
#!python -m spacy download es_core_news_lg

     ---------------------------------------- 0.0/568.0 MB ? eta -:--:--
     -------------------------------------- 0.0/568.0 MB 320.0 kB/s eta 0:29:35
     -------------------------------------- 0.1/568.0 MB 648.1 kB/s eta 0:14:37
     ---------------------------------------- 0.4/568.0 MB 3.0 MB/s eta 0:03:08
     ---------------------------------------- 0.9/568.0 MB 5.3 MB/s eta 0:01:48
     ---------------------------------------- 1.6/568.0 MB 7.2 MB/s eta 0:01:19
     ---------------------------------------- 2.5/568.0 MB 9.4 MB/s eta 0:01:00
     --------------------------------------- 3.8/568.0 MB 12.0 MB/s eta 0:00:47
     --------------------------------------- 5.4/568.0 MB 15.1 MB/s eta 0:00:38
      -------------------------------------- 7.6/568.0 MB 18.7 MB/s eta 0:00:30
      ------------------------------------- 10.5/568.0 MB 29.7 MB/s eta 0:00:19
      ------------------------------------- 10.5/568.0 MB 29.7 MB/s eta 0:00:19
      ------------------------------------- 12.

In [1]:
import pandas as pd
import requests
import datetime
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import string
import collections
import spacy
import ast
import unicodedata
from gensim import corpora
from collections import Counter
import copy

In [2]:
#Funciones para usar nltk
def tokenizar_texto(texto):
    tokens = nltk.word_tokenize(texto)
    tokens = [token.lower() for token in tokens if re.match(r'[a-z]+', token) and token not in string.punctuation]
    return tokens

def stopwords_texto(texto):
    tokens = nltk.word_tokenize(texto)
    tokens = [token.lower() for token in tokens if re.match(r'[a-z]+', token) and token not in string.punctuation]
    stop_words = set(stopwords.words('spanish'))
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

def lemmatizar_texto(texto):
    tokens = nltk.word_tokenize(texto)
    tokens = [token.lower() for token in tokens if re.match(r'[a-z]+', token) and token not in string.punctuation]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    stop_words = set(stopwords.words('spanish'))
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

#Funciones para usar spacy

def stopword_spacy(texto): # Tokens en minusculas, sin signos de puntuación, sólo letras y sin stopwords
    doc = nlp(texto.lower())
    tokens = [token.text for token in doc if not token.is_punct and token.is_alpha and not token.is_stop]
    return tokens
def lematizar_spacy(texto): # Tokens en minusculas, sin signos de puntuación, sólo letras y sin stopwords
    doc = nlp(texto.lower())
    tokens = [token.lemma_ for token in doc if not token.is_punct and token.is_alpha and not token.is_stop]
    return tokens

#Otras funciones

#Contar palabras
def count_words(text):
    words = re.findall(r'\b\w+\b',text)
    return len(words)

#Preprocesado: Eliminar acentos
def quitar_acentos(texto):
    tokens_sin_acentos = []
    for token in texto:
        if 'ñ' in token or 'Ñ' in token:
            tokens_sin_acentos.append(token)
        else:
            normalized_token = unicodedata.normalize('NFKD', token)
            token_sin_acento = ''.join([c for c in normalized_token if unicodedata.category(c) != 'Mn'])
            tokens_sin_acentos.append(token_sin_acento)
    return tokens_sin_acentos

#Contar textos cortos
def contar_tokens_cortos(texto):
    contador = 0
    for token in texto:
        if len(token) < 2:
            contador += 1
    return contador

#Obtener los tokens cortos
def filtrar_tokens_cortos(texto):
    tokens_cortos = []
    for token in texto:
        if len(token) < 2:
            tokens_cortos.append(token)
    if not tokens_cortos:  
        return None  
    else:
        return tokens_cortos
    
#Preprocesado eliminar los tokens que tiene un solo caracter
def eliminar_tokens_un_caracter(texto):
    tokens = [token for token in texto if len(token) > 1]
    return tokens   

#Mostrar palabras más comunes en el corpus
def palabras_mas_comunes(texto, num_palabras=10):

    frecuencia_total = Counter()
    for lista in texto:
        frecuencia_total.update(lista)
    # Imprimir el número de tokens únicos
    print("\nNúmero de tokens únicos:", len(frecuencia_total))    
    # Imprimir las n palabras más comunes
    print(f"Las {num_palabras} palabras más comunes:")
    for palabra, frecuencia in frecuencia_total.most_common(num_palabras):
        print(f"{palabra}: {frecuencia}")
    return frecuencia_total  

#Dado un token buscar los documentos que lo tienen
def buscar_documentos(documentos, token_buscar):
        documentos_con_token = []
        for i, documento in enumerate(documentos):
            if token_buscar in documento:
                documentos_con_token.append(i)  # Guarda el índice del documento
        return documentos_con_token,token_buscar

In [3]:
df_original = pd.read_csv('qys_original.csv', sep=";")

In [4]:
# Creamos otro dataFrame para trabajar los textos
df_word = df_original.copy()

## 1.- Unir titulo y texto

In [5]:
# Creamos el campo texto como concatenación de título y descripción
df_word['texto'] = df_word['title'].astype(str) + ' ' + df_word['description'].astype(str)

In [6]:
# Creamos campo con el numero de palabras
df_word['count_word'] = df_word['texto'].apply(count_words)

## 2.- Textos cortos

In [38]:
# Mostrar textos cortos
textos_cortos = df_word[df_word['count_word'] == 3]
textos_cortos['texto']

21315            Poda Sin comentarios
23878          Baldosa rota   Agujero
47062            Adopción animal Gato
49745            Roto Rotura columpio
65249    semáforo fundido  ¿¿ findido
Name: texto, dtype: object

In [10]:
# Mostrar textos cortos
textos_cortos = df_word[df_word['count_word'] < 5]
textos_cortos['texto']

3276                                prueba3 Prueba3
4104     carteles pornograficos   Espectáculo hole 
4283          Incidencia Adjunto archivo solicitado
21313                                     Poda Poda
21315                          Poda Sin comentarios
23036          Apoyo participacion Colegio vadorrey
23878                        Baldosa rota   Agujero
26613                   Papelera rota Papelera rota
28926                   Árbol partido Árbol partido
34761                Limpieza calles Email anterior
35832                       Señal caida Señal caida
38190                     Baldosa rota Baldosa rota
38392                     Baldosa rota Baldosa rota
38512                     Baldosa rota Baldosa rota
47062                          Adopción animal Gato
47578                 Farola fundida Farola fundida
49358                 Basuras, escombros Nueva foto
49359                 Basuras, escombros Nueva foto
49360                 Basuras  escombros Nueva foto
49745       

In [8]:
#eliminamos los documentos con menos de dos palabras
df_word=df_word[df_word['count_word'] >= 3]

## NLTK 

In [17]:
df_preprocesado_nltk = df_word.copy()
df_preprocesado_nltk['text_token'] = df_preprocesado_nltk['texto'].apply(tokenizar_texto)
df_preprocesado_nltk['text_stop'] = df_preprocesado_nltk['texto'].apply(stopwords_texto)
df_preprocesado_nltk['text_lema'] = df_preprocesado_nltk['texto'].apply(lemmatizar_texto)

In [20]:
df_preprocesado_nltk['text_lema']

0        [equipo, flip-flap, dirijo, ustedes, aprovecha...
1        [instalación, deportiva, gimnasia, deportiva, ...
2        [duda, permisos, hola, escribo, nombre, grupo,...
3        [noches, jueves, viernes, sábado, cada, vez, i...
4        [falta, instalaciones, deportivas, familia, am...
                               ...                        
66452    [reposición, reparación, cubo, basura, buenos,...
66453    [baldosas, defectuosas, justo, puerta, viviend...
66454    [aviso, reclamación, sigue, respuesta, buenos,...
66455    [recogida, basura, buenos, dia, muestro, males...
66456    [residencia, xiort, pontoneros-, madre, rafols...
Name: text_lema, Length: 66455, dtype: object

In [ ]:
df_preprocesado_nltk.to_csv('texto_procesado_nltk.csv', sep=';', index=False)

In [9]:
df_preprocesado_spacy = df_word.copy()

## SPACY

In [10]:
# Cargar el modelo de spaCy para el idioma español
nlp = spacy.load("es_core_news_lg")
df_preprocesado_spacy['stopword_spacy'] = df_preprocesado_spacy['texto'].apply(stopword_spacy)


In [11]:
df_preprocesado_spacy['stopword_spacy']

0        [equipo, dirijo, aprovechando, servicio, dejar...
1        [instalación, deportiva, gimnasia, deportiva, ...
2        [duda, permisos, hola, escribo, nombre, grupo,...
3        [noches, jueves, viernes, sábado, imposible, d...
4        [falta, instalaciones, deportivas, familia, am...
                               ...                        
66452    [reposición, reparación, cubo, basura, agradec...
66453    [baldosas, defectuosas, justo, puerta, viviend...
66454    [aviso, reclamación, respuesta, puse, queja, c...
66455    [recogida, basura, muestro, malestar, contened...
66456    [residencia, xiort, madre, rafols, dirigir, de...
Name: stopword_spacy, Length: 66455, dtype: object

In [12]:
df_preprocesado_spacy['texto_procesado_spacy'] = df_preprocesado_spacy['texto'].apply(lematizar_spacy)

In [16]:
df_preprocesado_spacy['texto_sin_acentos']

0        [equipo, dirigir, aprovechar, servicio, dejar,...
1        [instalacion, deportivo, gimnasia, deportivo, ...
2        [dudar, permiso, hola, escribir, nombre, grupo...
3        [noche, jueves, viernes, sabado, imposible, do...
4        [falta, instalacion, deportivo, familia, amant...
                               ...                        
66452    [reposicion, reparacion, cubo, basura, agradec...
66453    [baldosa, defectuoso, justo, puerta, vivienda,...
66454    [aviso, reclamacion, respuesta, poner, queja, ...
66455    [recogida, basura, mostrar, malestar, contened...
66456    [residencia, xiort, madre, rafols, dirigir, de...
Name: texto_sin_acentos, Length: 66455, dtype: object

In [ ]:
df_preprocesado_spacy['texto_procesado_spacy'] = df_preprocesado_spacy['texto_procesado_spacy'].apply(ast.literal_eval)
df_preprocesado_spacy['stopword_spacy'] = df_preprocesado_spacy['stopword_spacy'].apply(ast.literal_eval)

## Quitar acentos 

In [15]:
df_preprocesado_spacy['texto_sin_acentos'] = df_preprocesado_spacy['texto_procesado_spacy'].apply(quitar_acentos)
df_preprocesado_spacy['stopword_sin_acentos'] = df_preprocesado_spacy['stopword_spacy'].apply(quitar_acentos)

In [19]:
documentos_filtrados = df_preprocesado_spacy['texto_sin_acentos'].apply(filtrar_tokens_cortos)

In [29]:
# Creamos campo con el numero de palabras
df_preprocesado_spacy['count_word_1'] = df_preprocesado_spacy['texto_procesado_spacy'].apply(contar_tokens_cortos)

In [31]:
df_preprocesado_spacy['count_word_1'].sum()

8377

## Eliminar tokens de 1 caracter 

In [17]:
df_preprocesado_spacy['texto_sin_cortos'] = df_preprocesado_spacy['stopword_sin_acentos'].apply(eliminar_tokens_un_caracter)
df_preprocesado_spacy['texto_sin_cortos_lema'] = df_preprocesado_spacy['texto_sin_acentos'].apply(eliminar_tokens_un_caracter)

In [111]:
df_preprocesado_spacy['texto_sin_cortos'].apply(contar_tokens_cortos).sum()

0

In [129]:
df_preprocesado_spacy.to_csv('texto_procesado_spacyv2.csv', sep=';', index=False)

In [23]:
df_preprocesado_nltk = pd.read_csv('texto_procesado_nltk.csv', sep=";")

In [25]:
df_preprocesado_nltk['text_stop'] = df_preprocesado_nltk['text_stop'].apply(ast.literal_eval)
df_preprocesado_nltk['text_lema']= df_preprocesado_nltk['text_lema'].apply(ast.literal_eval)
df_preprocesado_nltk['text_token'] = df_preprocesado_nltk['texto'].apply(tokenizar_texto)
df_preprocesado_nltk['text_stop'] = df_preprocesado_nltk['texto'].apply(stopwords_texto)
df_preprocesado_nltk['text_lema'] = df_preprocesado_nltk['texto'].apply(lemmatizar_texto)
df_preprocesado_nltk['stopword_sin_acentos'] = df_preprocesado_nltk['text_stop'].apply(quitar_acentos)
df_preprocesado_nltk['lema_sin_acentos'] = df_preprocesado_nltk['text_lema'].apply(quitar_acentos)
df_preprocesado_nltk['texto_sin_cortos_stop'] = df_preprocesado_nltk['stopword_sin_acentos'].apply(eliminar_tokens_un_caracter)
df_preprocesado_nltk['texto_sin_cortos_lema'] = df_preprocesado_nltk['lema_sin_acentos'].apply(eliminar_tokens_un_caracter)

## Generar diccionario

In [29]:
dictionary_nltk_stop = corpora.Dictionary(df_preprocesado_nltk['texto_sin_cortos_stop'])
dictionary_nltk_lema = corpora.Dictionary(df_preprocesado_nltk['texto_sin_cortos_lema'])

In [30]:
len(dictionary_nltk_stop)

68224

In [34]:
dictionary_nltk_stop.save('dictionary_nltk_stop.gensim')

In [ ]:
# Cargar el diccionario
loaded_dictionary = corpora.Dictionary.load('dictionary_nltk_stop.gensim')

In [32]:
len(dictionary_nltk_lema)

67777

In [33]:
df_preprocesado_nltk['texto_sin_cortos_lema']

0        [dirijo, ustedes, aprovechando, servicio, deja...
1        [deportiva, gimnasia, deportiva, padre, chaval...
2        [escribo, nombre, grupo, numeroso, amigo, cual...
3        [viernes, sabado, vez, imposible, dormir, desc...
4        [instalaciones, deportivas, familia, amante, d...
                               ...                        
66450    [reparacion, cubo, basura, dias, agradeceremos...
66451    [defectuosas, puerta, vivienda, baldosas, hund...
66452    [reclamacion, sigue, respuesta, dias, hace, di...
66453    [basura, dia, muestro, malestar, ver, contened...
66454    [xiort, pontoneros-, madre, dias, si, dirigir,...
Name: texto_sin_cortos_lema, Length: 66455, dtype: object

In [32]:
df_preprocesado_spacy = pd.read_csv('texto_procesado_spacyv2.csv', sep=";")

In [33]:
df_preprocesado_spacy['texto_sin_cortos'] = df_preprocesado_spacy['texto_sin_cortos'].apply(ast.literal_eval)
df_preprocesado_spacy['texto_sin_cortos_lema'] = df_preprocesado_spacy['texto_sin_cortos_lema'].apply(ast.literal_eval)

In [35]:
dictionary_spacy_stop = corpora.Dictionary(df_preprocesado_spacy['texto_sin_cortos'])

In [36]:
len(dictionary_spacy_stop)

68459

In [18]:
dictionary_spacy_lema = corpora.Dictionary(df_preprocesado_spacy['texto_sin_cortos_lema'])

In [19]:
len(dictionary_spacy_lema)

50127

In [44]:
dictionary_spacy_stop.save('dictionary_spacy_stop.gensim')

In [45]:
dictionary_spacy_lema.save('dictionary_spacy_lema.gensim')

In [20]:
frecuencia_spacy=palabras_mas_comunes(df_preprocesado_spacy['texto_sin_cortos_lema'],20)


Número de tokens únicos: 50127
Las 20 palabras más comunes:
calle: 53881
zona: 22554
parque: 18194
gracia: 14200
saludo: 12831
acera: 12609
solicitar: 12440
barrio: 11081
arbol: 10469
semaforo: 10304
pasar: 10298
año: 10237
contenedor: 10085
zaragoza: 9517
plaza: 9348
avenida: 8984
basura: 8428
gracias: 8422
persona: 8413
vecino: 8371


In [21]:
frecuencia_spacy["dias"]

2

In [109]:
frecuencia_nltk=palabras_mas_comunes(df_preprocesado_nltk['texto_sin_cortos_stop'],20)


Número de tokens únicos: 68224
Las 20 palabras más comunes:
calle: 41614
dias: 18018
zona: 16863
si: 14633
parque: 11109
hace: 10717
saludo: 10089
gracias: 9328
barrio: 8795
acera: 8104
dia: 8010
semaforo: 6827
dos: 6803
avenida: 6739
favor: 6656
vecinos: 6616
paso: 6549
años: 6266
puede: 6263
agua: 6239


In [110]:
frecuencia_nltk["semaforo"]

6827

In [114]:
frecuencia_spacy["queja"]

6811

In [97]:
frecuencia_nltk["calle"]

48066

In [115]:
frecuencia_nltk["solicito"]

1433

In [116]:
 # Filtrar palabras que aparecen:
    # - Menos de 2 veces en los documentos.
    # - En más del 0.5 (50%) de los documentos.
    # keep_n: Mantener solo las 5 palabras más frecuente
dictionary_spacy_stop.filter_extremes(no_below=2, no_above=0.7, )

In [177]:
list(sorted(dictionary_spacy_stop.token2id.keys()))

['aa',
 'aaa',
 'aaaaaasco',
 'aaaaah',
 'aaaaalfonso',
 'aaaahhh',
 'aaahhh',
 'aacerque',
 'aademas',
 'aafaltar',
 'aai',
 'aaierta',
 'aaim',
 'aaium',
 'aaiun',
 'aal',
 'aala',
 'aalcaldesa',
 'aandador',
 'aapecto',
 'aapra',
 'aaragon',
 'aarancada',
 'aarece',
 'aarreglo',
 'aavv',
 'ab',
 'aba',
 'abad',
 'abada',
 'abadesas',
 'abadia',
 'abajar',
 'abajo',
 'abalanzado',
 'abalanzandose',
 'abalanzaron',
 'abalanzo',
 'abaldosada',
 'abalorios',
 'abalos',
 'abanderamiento',
 'abandobo',
 'abandomada',
 'abandon',
 'abandona',
 'abandonaban',
 'abandonad',
 'abandonada',
 'abandonadas',
 'abandonadass',
 'abandonadicos',
 'abandonadisimo',
 'abandonado',
 'abandonados',
 'abandonadosimas',
 'abandonamos',
 'abandonan',
 'abandonando',
 'abandonaos',
 'abandonar',
 'abandonara',
 'abandonaramos',
 'abandonaran',
 'abandonarla',
 'abandonarlas',
 'abandonarlo',
 'abandonarlos',
 'abandonaron',
 'abandonas',
 'abandonasa',
 'abandonasen',
 'abandonda',
 'abandondada',
 'abando

In [130]:
# Iterar sobre el diccionario dfs
for token_id, frecuencia in dictionary_spacy_stop.dfs.items():
    token = dictionary_spacy_stop.id2token[token_id]
    print(f"Token: {token}, ID: {token_id}, Frecuencia: {frecuencia}")

Token: equipo, ID: 28, Frecuencia: 259
Token: dirijo, ID: 22, Frecuencia: 927
Token: aprovechando, ID: 5, Frecuencia: 179
Token: servicio, ID: 74, Frecuencia: 3670
Token: dejar, ID: 19, Frecuencia: 1343
Token: momento, ID: 44, Frecuencia: 1413
Token: gimnasia, ID: 36, Frecuencia: 81
Token: deportiva, ID: 20, Frecuencia: 177
Token: zaragoza, ID: 85, Frecuencia: 7412
Token: nombre, ID: 54, Frecuencia: 722
Token: hijo, ID: 39, Frecuencia: 521
Token: años, ID: 7, Frecuencia: 5320
Token: pertenece, ID: 58, Frecuencia: 213
Token: club, ID: 14, Frecuencia: 176
Token: llevan, ID: 41, Frecuencia: 1290
Token: agradecimientos, ID: 2, Frecuencia: 10
Token: gestion, ID: 35, Frecuencia: 544
Token: problema, ID: 60, Frecuencia: 4170
Token: entrenan, ID: 25, Frecuencia: 24
Token: espacio, ID: 29, Frecuencia: 2290
Token: super, ID: 79, Frecuencia: 160
Token: reducido, ID: 65, Frecuencia: 141
Token: cad, ID: 9, Frecuencia: 4
Token: comparten, ID: 16, Frecuencia: 16
Token: chicas, ID: 13, Frecuencia: 51


In [27]:
# Obtener el mapeo id -> palabra
token2word = {id_: word for word, id_ in dictionary_spacy_lema.token2id.items()}

# Obtener la frecuencia de cada palabra
token_frequencies = dictionary_spacy_lema.cfs  # cfs (collection frequency statistics)

# Convertirlo en un DataFrame ordenado
df = pd.DataFrame(list(token_frequencies.items()), columns=["token_id", "frequency"])
df["word"] = df["token_id"].map(token2word)  # Mapear id a palabra

# Ordenar de mayor a menor frecuencia
df = df.sort_values(by="frequency", ascending=False).reset_index(drop=True)

print(df)


       token_id  frequency          word
0           167      53881         calle
1           420      22554          zona
2           636      18194        parque
3            34      14200        gracia
4            65      12831        saludo
...         ...        ...           ...
50122     29193          1    metiendola
50123     29190          1  hacinandolas
50124     29194          1      precinta
50125     29186          1         bucar
50126     50126          1         xiort

[50127 rows x 3 columns]


In [28]:
df[df["word"]=="dias"]

,token_id,frequency,word
19418,27419,2,dias


In [29]:
# Obtener el mapeo id -> palabra
token2word = {id_: word for word, id_ in dictionary_spacy_lema.token2id.items()}

# Obtener la frecuencia de cada palabra
token_frequencies = dictionary_spacy_lema.dfs  # cfs (collection frequency statistics)

# Convertirlo en un DataFrame ordenado
df_documentos = pd.DataFrame(list(token_frequencies.items()), columns=["token_id", "frequency"])
df_documentos["word"] = df_documentos["token_id"].map(token2word)  # Mapear id a palabra

# Ordenar de mayor a menor frecuencia
df_documentos = df_documentos.sort_values(by="frequency", ascending=False).reset_index(drop=True)

print(df_documentos)

       token_id  frequency          word
0           167      28020         calle
1            34      13952        gracia
2           420      13121          zona
3            65      12784        saludo
4           346      11653     solicitar
...         ...        ...           ...
50122     29018          1  metropolitar
50123     29016          1       golpeen
50124     29014          1     bambolean
50125     29013          1     deportita
50126     50126          1         xiort

[50127 rows x 3 columns]


In [30]:
df_documentos[df_documentos["word"]=="dias"]

,token_id,frequency,word
20835,27419,2,dias


In [31]:
# Crear una copia del diccionario para no modificar el original.
dictionary_filtrado = copy.deepcopy(dictionary_spacy_lema)

# Aplicar filter_extremes
dictionary_filtrado.filter_extremes(no_below=7, no_above=0.7)

In [32]:
# Obtener el mapeo id -> palabra
token2word = {id_: word for word, id_ in dictionary_filtrado.token2id.items()}

# Obtener la frecuencia de cada palabra
token_frequencies = dictionary_filtrado.dfs  # dfs (documents frequency statistics)

# Convertirlo en un DataFrame ordenado
df_documentos_filtrado = pd.DataFrame(list(token_frequencies.items()), columns=["token_id", "frequency"])
df_documentos_filtrado["word"] = df_documentos_filtrado["token_id"].map(token2word)  # Mapear id a palabra

# Ordenar de mayor a menor frecuencia
df_documentos_filtrado = df_documentos_filtrado.sort_values(by="frequency", ascending=False).reset_index(drop=True)

print(df_documentos_filtrado)

       token_id  frequency         word
0           158      28020        calle
1            33      13952       gracia
2           391      13121         zona
3            63      12784       saludo
4           320      11653    solicitar
...         ...        ...          ...
11738      7912          7     suroeste
11739      8602          7      obligen
11740      8616          7      clienta
11741      4673          7   inhumacion
11742      9507          7  escuchar el

[11743 rows x 3 columns]


In [33]:
df_documentos_filtrado[:10]

,token_id,frequency,word
0,158,28020,calle
1,33,13952,gracia
2,391,13121,zona
3,63,12784,saludo
4,320,11653,solicitar
5,591,9144,parque
6,189,8301,gracias
7,362,8038,pasar
8,331,7714,acera
9,393,7648,barrio


In [37]:
list(sorted(df_documentos[df_documentos["frequency"]==7]))

['frequency', 'token_id', 'word']

In [39]:

# Crear una copia del diccionario para no modificar el original.
dictionary_filtrado = copy.deepcopy(dictionary_spacy_lema)

# Aplicar filter_extremes
dictionary_filtrado.filter_extremes(no_below=2, no_above=0.5)

# Obtener los IDs de tokens originales
tokens_originales = set(dictionary_spacy_stop.token2id.values())

# Obtener los IDs de tokens filtrados
tokens_filtrados = set(dictionary_filtrado.token2id.values())

# Calcular la diferencia
tokens_eliminados = tokens_originales - tokens_filtrados

# Eliminar los tokens de los textos
textos_filtrados = []
for texto in df_preprocesado_spacy['texto_sin_cortos']:
    texto_filtrado = [token for token in texto if dictionary_spacy_stop.token2id.get(token) not in tokens_eliminados]
    textos_filtrados.append(texto_filtrado)


In [47]:
df_preprocesado_spacy['texto_sin_tokens']=textos_filtrados

In [153]:
len(df_preprocesado_spacy['texto_sin_tokens'][2])

41

In [154]:
len(df_preprocesado_spacy['texto_sin_cortos'][2])

41

In [49]:
print(df_preprocesado_spacy.loc[11742, ["texto_sin_cortos", "texto_sin_tokens"]])

texto_sin_cortos    [garrafa, aceite, motor, hola, queja, quiero, ...
texto_sin_tokens    [garrafa, aceite, motor, hola, queja, quiero, ...
Name: 11742, dtype: object


In [190]:
tokens_eliminados

{36646,
 36647,
 36648,
 36649,
 36650,
 36651,
 36652,
 36653,
 36654,
 36655,
 36656,
 36657,
 36658,
 36659,
 36660,
 36661,
 36662,
 36663,
 36664,
 36665,
 36666,
 36667,
 36668,
 36669,
 36670,
 36671,
 36672,
 36673,
 36674,
 36675,
 36676,
 36677,
 36678,
 36679,
 36680,
 36681,
 36682,
 36683,
 36684,
 36685,
 36686,
 36687,
 36688,
 36689,
 36690,
 36691,
 36692,
 36693,
 36694,
 36695,
 36696,
 36697,
 36698,
 36699,
 36700,
 36701,
 36702,
 36703,
 36704,
 36705,
 36706,
 36707,
 36708,
 36709,
 36710,
 36711,
 36712,
 36713,
 36714,
 36715,
 36716,
 36717,
 36718,
 36719,
 36720,
 36721,
 36722,
 36723,
 36724,
 36725,
 36726,
 36727,
 36728,
 36729,
 36730,
 36731,
 36732,
 36733,
 36734,
 36735,
 36736,
 36737,
 36738,
 36739,
 36740,
 36741,
 36742,
 36743,
 36744,
 36745,
 36746,
 36747,
 36748,
 36749,
 36750,
 36751,
 36752,
 36753,
 36754,
 36755,
 36756,
 36757,
 36758,
 36759,
 36760,
 36761,
 36762,
 36763,
 36764,
 36765,
 36766,
 36767,
 36768,
 36769,
 36770,


In [178]:
list(sorted(dictionary_filtrado.token2id.keys()))

['aa',
 'aaa',
 'aacerque',
 'aaiun',
 'aal',
 'aaragon',
 'aavv',
 'abad',
 'abadia',
 'abajo',
 'abalanzado',
 'abalanzo',
 'abandona',
 'abandonad',
 'abandonada',
 'abandonadas',
 'abandonado',
 'abandonados',
 'abandonan',
 'abandonando',
 'abandonaos',
 'abandonar',
 'abandonarlo',
 'abandonarlos',
 'abandonaron',
 'abandonas',
 'abandonda',
 'abandone',
 'abandonen',
 'abandono',
 'abandonos',
 'abanico',
 'abanto',
 'abaratando',
 'abaratar',
 'abarca',
 'abarcan',
 'abarcando',
 'abarcar',
 'abarque',
 'abarrotada',
 'abarrotadas',
 'abarrotado',
 'abarrotados',
 'abarrotan',
 'abastece',
 'abastecen',
 'abastecer',
 'abastecernos',
 'abastecerse',
 'abastecimiento',
 'abasto',
 'abastos',
 'abatible',
 'abb',
 'abdomen',
 'abdominales',
 'abdulas',
 'abecedario',
 'abejar',
 'abejas',
 'abejeros',
 'aben',
 'abenoza',
 'aberracion',
 'aberraciones',
 'aberrante',
 'aberto',
 'abertura',
 'aberturas',
 'abeto',
 'abetos',
 'abierta',
 'abiertamente',
 'abiertas',
 'abierto',
 

In [25]:
sorted((df_preprocesado_spacy['texto_sin_cortos_lema'][64547]))

['ayto',
 'codigo',
 'compra',
 'comprar',
 'cultura',
 'descuento',
 'descuento',
 'descuento',
 'descuento',
 'enlace',
 'enlace',
 'enlace',
 'enlace',
 'enlace',
 'entrada',
 'funcionar',
 'ibercaja',
 'ibercaja',
 'ingresar',
 'newsletter',
 'noticia',
 'obtener',
 'perdido',
 'prometer',
 'promocional',
 'seleccionar',
 'tarifa',
 'visitar',
 'web',
 'web',
 'web',
 'zgz',
 'zgzcultura']